In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


# Load historical stock data
data = pd.read_csv('data/us_stock/TSLA.csv', parse_dates=['Date'], index_col='Date')

data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,9.986667,10.165333,9.770000,10.006667,10.006667,92826000
2014-01-03,10.000000,10.146000,9.906667,9.970667,9.970667,70425000
2014-01-06,10.000000,10.026667,9.682667,9.800000,9.800000,80416500
2014-01-07,9.841333,10.026667,9.683333,9.957333,9.957333,75511500
2014-01-08,9.923333,10.246667,9.917333,10.085333,10.085333,92448000


In [2]:
# Create features and labels
data['SMA_10'] = data['Close'].rolling(window=10).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['Momentum'] = data['Close'] - data['Close'].shift(10)
data.dropna(inplace=True)

# Label: 1 if the price goes up the next day, 0 otherwise
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)

data.head()
                                                        

,Open,High,Low,Close,Adj Close,Volume,SMA_10,SMA_50,Momentum,Target
Date,,,,,,,,,,
2014-03-14,15.686000,15.796000,15.221333,15.398000,15.398000,124345500,16.271400,13.066987,-0.922668,1
2014-03-17,15.663333,15.862000,15.366667,15.598667,15.598667,88689000,16.160866,13.178827,-1.105333,1
2014-03-18,15.796667,16.100000,15.668000,16.002666,16.002666,93634500,16.062200,13.299467,-0.986666,0
2014-03-19,16.092667,16.103333,15.567333,15.722667,15.722667,76069500,15.950067,13.417920,-1.121333,0
2014-03-20,15.744000,15.950000,15.557333,15.660667,15.660667,57268500,15.829867,13.531987,-1.202000,0


In [3]:
# Features and target
X = data[['SMA_10', 'SMA_50', 'Momentum']]
y = data['Target']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Save the scaler and model
import joblib
joblib.dump(scaler, 'model/scaler.pkl')
joblib.dump(model, 'model/logistic_model.pkl')

Accuracy: 0.48880597014925375


['model/logistic_model.pkl']

In [4]:
import warnings
warnings.filterwarnings("ignore")

from trading.strategy import *
from trading.trader import *

class MLTradingStrategy(bt.Strategy):
    """
    A strategy that uses a machine learning model to predict buy/sell signals.
    """
    def __init__(self):
        # Load the pre-trained model and scaler
        self.model = joblib.load('model/logistic_model.pkl')
        self.scaler = joblib.load('model/scaler.pkl')
        
        # Initialize indicators
        self.sma_10 = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
        self.sma_50 = bt.indicators.SimpleMovingAverage(self.data.close, period=50)
        self.momentum = self.data.close - self.data.close(-10)
        
    def next(self):
        # Create feature set for prediction
        features = [[self.sma_10[0], self.sma_50[0], self.momentum[0]]]
        features = self.scaler.transform(features)  # Scale features

        # Get prediction
        prediction = self.model.predict(features)[0]  # 1 for buy, 0 for sell

        if not self.position:  # Not in a position
            if prediction == 1:
                self.buy()
                self.log(f'BUY CREATE {self.data.close[0]:.2f}')
        else:  # In a position
            if prediction == 0:
                self.sell()
                self.log(f'SELL CREATE {self.data.close[0]:.2f}')
    
    def log(self, txt, dt=None):
        """Logging function for this strategy"""
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} - {txt}')

In [5]:
# Initialize the AITrader
trader = AITrader(start_date="2020-01-01", end_date="2021-01-01")

# Set your desired strategy; for example, using the BuyHoldStrategy
trader.add_strategy(MLTradingStrategy)

# Run the backtest
trader.run(1, stock_ticker="AAPL")


--- AITrader initialization ---
Strategy 'MLTradingStrategy' added without parameters.
Data loaded.
Starting Value: 1000000
Sizer set to 95%.
Analyzers added.

--- Backtesting ---
2020-03-13 - BUY CREATE 69.49
Ending value: 1985869
Total Returns: 0.69
Annualized Returns: 0.98
Max Drawdown: 18.63%

--- Strategy Evaluation ---
Total Returns are positive, indicating a profitable strategy.
Max Drawdown is within acceptable limits (< 20%), suggesting a stable strategy.
